In [1]:
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import requests
import time
from joblib import Parallel, delayed
from tqdm import tqdm
import re


Web Scraping, part 1

In [2]:
link = "https://ic2s2-2023.org/program"
r = requests.get(link)
soup = BeautifulSoup(r.content)

table = soup.find("table",{"class":"tutorials"})
table_rows = table.find_all("tr")

#HERE I GET THE HEADER
ths = table_rows[0].find_all('th')
header = [th.text.replace("\n","") for th in ths]
    
#HERE I GET THE ROWS
rows = []
for tr in table_rows[1:]:
    tds = tr.find_all('td')
    for td in tds:
        td = td.text.replace("\n","")
        if "Keynote" in td:
            td = td.replace("Keynote - ","")
            rows.append(td)    

content = soup.find_all("ul",{"class":"nav_list"})
for nav in content:
    content_rows = nav.find_all("i")
    for c_row in content_rows:
        td = c_row.text.replace("\n","")
        for item in td.split(","):
            item = item.strip()
            item = item.encode("utf-8", "ignore").decode("utf-8")
            # handling duplicates
            if item not in rows:
                rows.append(item)

link = "https://ic2s2-2023.org/program_committee"
r2 = requests.get(link)
soup2 = BeautifulSoup(r2.content)

extra_names = []
content = soup2.find_all("div",{"class":"container"})
for line in content:
    line_name = line.find_all("b")
    for name in line_name:
        name = name.text.replace("\n","")
        name = name.encode("utf-8", "ignore").decode("utf-8")
        if name not in rows:
            extra_names.append(name)

link = "https://ic2s2-2023.org/tutorials"
r3 = requests.get(link)
soup3 = BeautifulSoup(r3.content)

content = soup3.find_all("div",{"class":"container"})
for container in content:
    line_name = container.find_all("b")
    for name in line_name:
        name = name.text.replace("\n","")
        name = name.encode("utf-8", "ignore").decode("utf-8")
        if name not in rows and name not in extra_names:
            extra_names.append(name)


In [3]:
author_names2023 = rows + extra_names
len(author_names2023)

1649

Names from 2024 seminars:

In [4]:
rows_2024 = []
oral = pd.read_excel("IC2S2_2024_oral_panels.xlsx")["Presentation authors"]
poster = pd.read_excel("IC2S2_2024_posters.xlsx")["Poster authors"]
talks = pd.read_excel("IC2S2_2024_lightning_talks.xlsx")["Presentation authors"]
excel_list = [oral,poster,talks]
for file in excel_list:
    for row in file:
        for line in row.split(";"):
            name = line.split(",")[0]
            name = name.strip()
            if name not in rows_2024:
                rows_2024.append(name)
len(rows_2024)
rows_2024

['Hazem Ibrahim',
 'Nouar AlDahoul',
 'Talal Rahwan',
 'Yasir Zaki',
 'Alvin Zhou',
 'Danaë Metaxa',
 'Shengchun Huang',
 'Stephanie Wang',
 'Grant Schoenebeck',
 'Md Sanzeed Anwar',
 'Paramveer Dhillon',
 'Seonhye Noh',
 'Stuart Soroka',
 'Lisa Oswald',
 'Philipp Lorenz-Spreen',
 'Alexander Wenz',
 'Anna-Carolina Haensch',
 'Leah von der Heyde',
 'Hans William Alexander Hanley',
 'Jennifer Pan',
 'Yingdan Lu',
 'Chloe Ahn',
 'Junghyun Lim',
 'Ashton Anderson',
 'George Eilender',
 'Joshua Introne',
 'Una Joh',
 'Ceren Budak',
 'Julia Mendelsohn',
 'Patrick Gordon Wall',
 'Asael H Sorensen',
 'Asmeret Naugle',
 'Casey Doyle',
 'Dan J. Krofcheck',
 'Matt Sweitzer',
 'Christian Zingg',
 'Christoph Benedikt Gote',
 'Frank Schweitzer',
 'Giona Casiraghi',
 'Giuseppe Russo',
 'Luca Verginer',
 'Arnab Kumar Sarker',
 'Patrick Park',
 'Bruno Lepri',
 'Eduardo López',
 'Robin Dunbar',
 'Sam G B Roberts',
 'Simone Centellegher',
 'Valentin Vergara Hidd',
 'Balaraju BATTU',
 'Alex Pentland',
 'B

In [5]:
#Using "scrollable element" we can find a couple names with paranthesis left to be cleaned:
def clean_name(name):
    return re.sub(r"\s*\(.*?\)", "", name)

author_names_2024 = [clean_name(name) for name in rows_2024]
author_names_2024

['Hazem Ibrahim',
 'Nouar AlDahoul',
 'Talal Rahwan',
 'Yasir Zaki',
 'Alvin Zhou',
 'Danaë Metaxa',
 'Shengchun Huang',
 'Stephanie Wang',
 'Grant Schoenebeck',
 'Md Sanzeed Anwar',
 'Paramveer Dhillon',
 'Seonhye Noh',
 'Stuart Soroka',
 'Lisa Oswald',
 'Philipp Lorenz-Spreen',
 'Alexander Wenz',
 'Anna-Carolina Haensch',
 'Leah von der Heyde',
 'Hans William Alexander Hanley',
 'Jennifer Pan',
 'Yingdan Lu',
 'Chloe Ahn',
 'Junghyun Lim',
 'Ashton Anderson',
 'George Eilender',
 'Joshua Introne',
 'Una Joh',
 'Ceren Budak',
 'Julia Mendelsohn',
 'Patrick Gordon Wall',
 'Asael H Sorensen',
 'Asmeret Naugle',
 'Casey Doyle',
 'Dan J. Krofcheck',
 'Matt Sweitzer',
 'Christian Zingg',
 'Christoph Benedikt Gote',
 'Frank Schweitzer',
 'Giona Casiraghi',
 'Giuseppe Russo',
 'Luca Verginer',
 'Arnab Kumar Sarker',
 'Patrick Park',
 'Bruno Lepri',
 'Eduardo López',
 'Robin Dunbar',
 'Sam G B Roberts',
 'Simone Centellegher',
 'Valentin Vergara Hidd',
 'Balaraju BATTU',
 'Alex Pentland',
 'B

In [6]:
import requests
import time
from joblib import Parallel, delayed
from tqdm import tqdm

BASE_URL = "https://api.openalex.org/authors"
email = "s204120@dtu.dk"

#Only used for Week 2 name searching so code is relatively slow on purpose to avoid API errors
def get_author_data(name):
    try:
        response = requests.get(BASE_URL, params={'search': name, 'mailto': email}, timeout=10)
        response.raise_for_status()
        data = response.json()

        if 'results' in data and data['results']:
            author = data['results'][0]  # Take the first result
            institutions = author.get('last_known_institutions', [])
            country_code = institutions[0].get('country_code', 'N/A') if institutions else 'N/A'

            return {
                'id': author.get('id', 'N/A'),
                'display_name': author.get('display_name', 'N/A'),
                'works_api_url': author.get('works_api_url', 'N/A'),
                'h_index': author.get('summary_stats', {}).get('h_index', 0),
                'works_count': author.get('works_count', 0),
                'country_code': country_code
            }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {name}: {e}")
    
    return None  # Return None if an error occurs
#10 Authors and parallel jobs (theoretical max: 25 author, 10 jobs /s)
def process_in_batches(author_names, batch_size=10, delay=1.0):
    authors_data = []
    for i in tqdm(range(0, len(author_names), batch_size), desc="Fetching author data"):
        batch = author_names[i:i + batch_size]  # Take a batch of names
        results = Parallel(n_jobs=batch_size)(delayed(get_author_data)(name) for name in batch)
        authors_data.extend([author for author in results if author])
        time.sleep(delay)  # enforcing 10 API calls /s max

    return authors_data
    
#Generate API info on author_names_2024:
authors_data = process_in_batches(author_names_2024, batch_size=10, delay=1.0)

Fetching author data: 100%|██████████| 124/124 [04:18<00:00,  2.08s/it]


In [7]:
df = pd.DataFrame(authors_data)
#Remove rows with works outside of range [5,5000]
df_filtered = df[(df['works_count'] >= 5) & (df['works_count'] <= 5000)]
df_filtered.to_csv('author_names_plain.csv', index=False)

In [ ]:
#Generate list of author ID's for use in the IC2S2 datasets, API calls:
IC2S2_author_IDs = [ids[-11:] for ids in pd.read_csv("author_names_plain.csv").id.to_numpy()]

In [27]:
# Define Concepts
# can find all concept ID's at: https://api.openalex.org/concepts
concepts1 = ['C144024400', 'C15744967', 'C162324750', 'C17744445']
concepts2 = ['C33923547', 'C121332964', 'C41008148']
EMAIL = "s204120@dtu.dk"

# Function to generate OpenAlex API request URL
def URL_filter(author_ids):
    base_url = "https://api.openalex.org/works"
    author_filter = f"author.id:{'|'.join(author_ids)}"
    concepts1_filter = '|'.join(concepts1)
    concepts2_filter = '|'.join(concepts2)
    filters = (
        "?filter="
        f"{author_filter},"
        f"cited_by_count:>10,"
        f"concepts.id:({concepts1_filter}),"
        f"concepts.id:({concepts2_filter})"
    )
    
    return base_url + filters

# Function to fetch works for a given batch of authors
def fetch_author_works(author_ids, max_results=200, cursor="*"):
    all_works = []
    
    while cursor:
        url = URL_filter(author_ids)
        params = {
            'per_page': max_results, 
            'cursor': cursor,  
            'mailto': EMAIL,
        }

        response = requests.get(url, params=params)
        data = response.json()

        if 'results' in data and data['results']:
            all_works.extend(data['results'])
        cursor = data.get('meta', {}).get('next_cursor', None)

    return all_works

# Function to extract relevant details from works
def extract_work_details(works):
    works_data = []
    for work in works:
        work_info = {
            'id': work.get('id', 'N/A'),
            'publication_year': work.get('publication_year', 'N/A'),
            'cited_by_count': work.get('cited_by_count', 0),
            'author_ids': [author['author'].get('id', 'N/A') for author in work.get('authorships', [])],
            'title': work.get('title', 'N/A'),
            'abstract_inverted_index': work.get('abstract_inverted_index', 'N/A'),
        }
        works_data.append(work_info)
    return works_data


def fetch_and_process_batch(chunk):
    works = fetch_author_works(chunk)
    return extract_work_details(works)

#Batch size = how many authors per API request, n_jobs are parallel threads.
def Request_batching(author_ids, batch_size=25, n_jobs=9):
    chunks = [author_ids[i:i + batch_size] for i in range(0, len(author_ids), batch_size)]
    
    all_works_data = []
    with tqdm(total=len(chunks), desc="Fetching batches") as pbar:
        for i in range(0, len(chunks), n_jobs):  # Process in groups of 10
            batch_chunks = chunks[i:i + n_jobs]  # Take up to 10 batches
            results = Parallel(n_jobs=n_jobs)(
                delayed(fetch_and_process_batch)(chunk) for chunk in batch_chunks
            )
            
            for batch in results:
                all_works_data.extend(batch)
            
            pbar.update(len(batch_chunks))  
            time.sleep(1)  

    return all_works_data
#Parent function to call the API using the ID list:
all_works_data = Request_batching(IC2S2_author_IDs)

df = pd.DataFrame(all_works_data)

Fetching batches: 100%|██████████| 41/41 [00:48<00:00,  1.19s/it]


Separating into the two datasets:

In [33]:
df_papers = pd.DataFrame(all_works_data, columns=['id', 'publication_year', 'cited_by_count', 'author_ids'])
df_abstracts = pd.DataFrame(all_works_data, columns=['id', 'title', 'abstract_inverted_index'])
df_papers.to_csv("IC2S2_Papers.csv", index=False)
df_abstracts.to_csv("IC2S2_Abstracts.csv", index=False)

The co-authors datasets:

In [36]:
#Grab all authors from author lists:
ls = []
for i in df_papers['author_ids']:
    ls.extend(i)
print(len(ls))
#Only unique authors:
unique_list = list(set([url.split('/')[-1] for url in ls]))
print(len(unique_list))

71361
28517


In [37]:
# Define Concepts
concepts1 = ['C144024400', 'C15744967', 'C162324750', 'C17744445']
concepts2 = ['C33923547', 'C121332964', 'C41008148']
EMAIL = "s204120@dtu.dk"

def URL_filter_authors(author_ids):
    """Generates OpenAlex API request URL for authors."""
    base_url = "https://api.openalex.org/authors"
    author_filter = f"id:{'|'.join(author_ids)}"
    filters = f"?filter={author_filter}"
    return base_url + filters

#Page limiting and cursor not relevant since we only ever fetch 25 authors.
def fetch_author_details(author_ids, max_results=200):
    """Fetches details of authors using OpenAlex API."""
    all_authors = []
    
    url = URL_filter_authors(author_ids)
    params = {'per_page': max_results, 'mailto': EMAIL}
    response = requests.get(url, params=params)
    data = response.json()
    
    if 'results' in data and data['results']:
        for author in data['results']:
            #Have to check CC beforehand because the list can be missing which breaks indexing
            institutions = author.get('last_known_institutions', [])
            #Empty list = False in python
            country_code = institutions[0].get('country_code', 'N/A') if institutions else 'N/A'
            author_info = {
                'id': author.get('id', 'N/A'),
                'display_name': author.get('display_name', 'N/A'),
                'works_api_url': author.get('works_api_url', 'N/A'),
                'h_index': author.get('summary_stats', {}).get('h_index', 0),
                'works_count': author.get('works_count', 0),
                'country_code': country_code
            }
            all_authors.append(author_info)
    
    return all_authors

def fetch_and_process_authors(chunk):
    """Fetch and process a batch of authors."""
    return fetch_author_details(chunk)

def Request_batching_authors(author_ids, batch_size=25, n_jobs=9):
    """Batch requests for author details."""
    chunks = [author_ids[i:i + batch_size] for i in range(0, len(author_ids), batch_size)]
    all_authors_data = []
    
    with tqdm(total=len(chunks), desc="Fetching author batches") as pbar:
        for i in range(0, len(chunks), n_jobs):
            batch_chunks = chunks[i:i + n_jobs]
            results = Parallel(n_jobs=n_jobs)(
                delayed(fetch_and_process_authors)(chunk) for chunk in batch_chunks
            )
            for batch in results:
                all_authors_data.extend(batch)
            pbar.update(len(batch_chunks))
            time.sleep(1)
    
    return all_authors_data

# Fetch co-author details excluding IC2S2 authors
coauthor_data = Request_batching_authors(unique_list)

Fetching author batches: 100%|██████████| 1141/1141 [05:36<00:00,  3.39it/s]


In [ ]:
IC2S2_Co_authors = pd.DataFrame(coauthor_data)
IC2S2_Co_authors.shape

(28517, 6)

In [ ]:
# Load the two CSVs into DataFrames
IC2S2_authors = pd.read_csv("author_names_plain.csv")  # Original authors
# Remove rows in coathors where the 'id' exists in authors
IC2S2_Co_authors = IC2S2_Co_authors[~IC2S2_Co_authors['id'].isin(IC2S2_authors['id'])]
# Apply same works_count filter range:
IC2S2_Co_authors = IC2S2_Co_authors[(IC2S2_Co_authors['works_count'] >= 5) & (IC2S2_Co_authors['works_count'] <= 5000)]

print(IC2S2_Co_authors.shape)

#Save filtered dataset:
IC2S2_Co_authors.to_csv("IC2S2_Co_authors.csv", index=False)


(25388, 6)


Finding the co-author works:

In [53]:
IC2S2_Co_authors_IDs = [ids[-11:] for ids in pd.read_csv("IC2S2_Co_authors.csv").id.to_numpy()]
print(IC2S2_Co_authors_IDs[0])
print(len(IC2S2_Co_authors_IDs))

A5083617223
25388


In [54]:
#Same code:::
concepts1 = ['C144024400', 'C15744967', 'C162324750', 'C17744445']
concepts2 = ['C33923547', 'C121332964', 'C41008148']
EMAIL = "s204120@dtu.dk"

# Function to generate OpenAlex API request URL
def URL_filter(author_ids):
    base_url = "https://api.openalex.org/works"
    author_filter = f"author.id:{'|'.join(author_ids)}"
    concepts1_filter = '|'.join(concepts1)
    concepts2_filter = '|'.join(concepts2)
    filters = (
        "?filter="
        f"{author_filter},"
        f"cited_by_count:>10,"
        f"concepts.id:({concepts1_filter}),"
        f"concepts.id:({concepts2_filter})"
    )
    
    return base_url + filters

# Function to fetch works for a given batch of authors
def fetch_author_works(author_ids, max_results=200, cursor="*"):
    all_works = []
    
    while cursor:
        url = URL_filter(author_ids)
        params = {
            'per_page': max_results, 
            'cursor': cursor,  
            'mailto': EMAIL,
        }

        response = requests.get(url, params=params)
        data = response.json()

        if 'results' in data and data['results']:
            all_works.extend(data['results'])
        cursor = data.get('meta', {}).get('next_cursor', None)

    return all_works

# Function to extract relevant details from works
def extract_work_details(works):
    works_data = []
    for work in works:
        work_info = {
            'id': work.get('id', 'N/A'),
            'publication_year': work.get('publication_year', 'N/A'),
            'cited_by_count': work.get('cited_by_count', 0),
            'author_ids': [author['author'].get('id', 'N/A') for author in work.get('authorships', [])],
            'title': work.get('title', 'N/A'),
            'abstract_inverted_index': work.get('abstract_inverted_index', 'N/A'),
        }
        works_data.append(work_info)
    return works_data


def fetch_and_process_batch(chunk):
    works = fetch_author_works(chunk)
    return extract_work_details(works)

#Batch size = how many authors per API request, n_jobs are parallel threads.
def Request_batching(author_ids, batch_size=25, n_jobs=9):
    chunks = [author_ids[i:i + batch_size] for i in range(0, len(author_ids), batch_size)]
    
    all_works_data = []
    with tqdm(total=len(chunks), desc="Fetching batches") as pbar:
        for i in range(0, len(chunks), n_jobs):  # Process in groups of 10
            batch_chunks = chunks[i:i + n_jobs]  # Take up to 10 batches
            results = Parallel(n_jobs=n_jobs)(
                delayed(fetch_and_process_batch)(chunk) for chunk in batch_chunks
            )
            
            for batch in results:
                all_works_data.extend(batch)
            
            pbar.update(len(batch_chunks))  
            time.sleep(1)  

    return all_works_data
#Parent function to call the API using the ID list:
all_works_data = Request_batching(IC2S2_Co_authors_IDs)

df = pd.DataFrame(all_works_data)

Fetching batches: 100%|██████████| 1016/1016 [24:14<00:00,  1.43s/it]


In [59]:
#split unfiltered into papers, abstracts
#remove from those all the works from author abs, pap, 
#in papers dataset, clean up author_id field
df.to_csv("IC2S2_Co_author_unfiltered.csv", index=False)
print(df.shape)

### Exclude works from the authors, in the co-authors dataset:
df1 = pd.read_csv("IC2S2_Papers.csv")
df2 = pd.read_csv("IC2S2_Co_author_unfiltered.csv")

df2_filtered = df2[~df2['id'].isin(df1['id'])]

print(df2_filtered.shape)

#Save filtered dataset:
df2_filtered.to_csv("IC2S2_Co_authors_filtered.csv", index=False)


#then go to step 4

(427454, 6)
(383510, 6)


In [ ]:
### REMOVES AUTHOR ID'S THAT AREN'T IN AUTHORS OR COAUTHORS

import concurrent.futures

# Load original authors and co-authors
df1 = pd.read_csv("author_names_plain.csv")  # Original authors
df2 = pd.read_csv("IC2S2_Co_authors.csv")  # Co-authors

# Create a set for fast lookups
author_id_filter = set(df1['id'].tolist() + df2['id'].tolist())

# Load the CSV with the 'author_ids' column
df_to_filter = pd.read_csv("IC2S2_Co_authors_filtered.csv")

# Function to filter out invalid author_ids using a set for faster lookups
def filter_author_ids(author_list):
    # Keep only those author IDs that are in the filter set
    return [author_id for author_id in author_list if author_id in author_id_filter]

# Function to process chunks, with error handling
def process_chunk(chunk):
    try:
        chunk.loc[:, 'author_ids'] = chunk['author_ids'].apply(lambda x: filter_author_ids(eval(x)))  # Assuming 'author_ids' is a stringified list
        return chunk
    except Exception as e:
        print(f"Error processing chunk: {e}")
        return chunk 

# Split DataFrame into chunks for parallel processing
chunk_size = 5000  # Adjust this depending on your system's memory capacity
chunks = [df_to_filter[i:i+chunk_size] for i in range(0, df_to_filter.shape[0], chunk_size)]

# Some magic to speed up the code:::
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(process_chunk, chunks))

# Combine the processed chunks back into a single DataFrame
df_to_filter_filtered = pd.concat(results)

# Lets save the work:
df_to_filter_filtered.to_csv("IC2S2_Co_authors_filtered_final.csv", index=False)


In [4]:

df_co_authors_papers = pd.DataFrame(df_to_filter_filtered, columns=['id', 'publication_year', 'cited_by_count', 'author_ids'])
df_co_authors_abstracts = pd.DataFrame(df_to_filter_filtered, columns=['id', 'title', 'abstract_inverted_index'])
df_co_authors_papers.to_csv("Co_authors_Papers.csv", index=False)
df_co_authors_abstracts.to_csv("Co_authors_Abstracts.csv", index=False)

Combining the datasets

In [ ]:
import pandas as pd

# Load the datasets (Adjust file paths as necessary)
df1 = pd.read_csv("author_names_plain.csv")  # Original authors
df2 = pd.read_csv("IC2S2_Co_authors.csv")  # Co-authors

df_papers = pd.read_csv("IC2S2_Papers.csv")  # Papers without co authors
df_coauthor_papers = pd.read_csv("Co_authors_Papers.csv")  # Co-authors papers dataset 

df_abstracts = pd.read_csv("IC2S2_Abstracts.csv")  # Abstracts without co authors
df_coauthor_abstracts = pd.read_csv("Co_authors_Abstracts.csv")  # Co-authors abstracts dataset

# --- 1. Combine Authors Dataset (remove duplicates) ---
df_authors_combined = pd.concat([df1, df2]).drop_duplicates(subset=['id'], keep='first')

# Save
df_authors_combined.to_csv("IC2S2_combined_authors.csv", index=False)

# --- 2. Combine Papers Dataset ---
df_papers_combined = pd.concat([df_papers, df_coauthor_papers])

# Remove papers with only one author by filtering based on 'author_ids' len
df_papers_combined = df_papers_combined[df_papers_combined['author_ids'].apply(lambda x: len(eval(x)) > 1)]

# Drop duplicates based on 'id'
df_papers_combined = df_papers_combined.drop_duplicates(subset=['id'], keep='first')

# Save 
df_papers_combined.to_csv("IC2S2_combined_papers.csv", index=False)

# --- 3. Combine Abstracts Dataset ---
df_abstracts_combined = pd.concat([df_abstracts, df_coauthor_abstracts]).drop_duplicates(subset=['id'], keep='first')

# Merge with df_papers_combined to retain only abstracts of papers with more than one author
df_abstracts_combined = df_abstracts_combined.merge(df_papers_combined[['id', 'author_ids']], on='id', how='inner')

# Apply the same author filtering logic
df_abstracts_combined = df_abstracts_combined[df_abstracts_combined['author_ids'].apply(lambda x: len(eval(x)) > 1)]

# Drop the 'author_ids' column, since abstracts don’t need it
df_abstracts_combined = df_abstracts_combined.drop(columns=['author_ids'])

# Save the cleaned abstracts dataset
df_abstracts_combined.to_csv("IC2S2_combined_abstracts.csv", index=False)
